In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("GOOGLE_API_KEY")


In [11]:
from google import genai
from google.genai import types

# Define the function declaration
weather_function = {
    "name": "get_current_temperature",
    "description": "Gets the current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name, e.g. San Francisco",
            },
        },
        "required": ["location"],
    },
}

def get_current_temperature(location):
    return {"location": location, "temperature": "21°C"}

# Client setup
client = genai.Client()

# tools object, which contains one or more function declarations
tools = types.Tool(function_declarations=[weather_function])
config = types.GenerateContentConfig(tools=[tools])

# Step 1: User asks a question
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[types.Content(role="user", parts=[types.Part(text="What's the temperature in Bengaluru?")])],
    config=config,
)

tool_call = response.candidates[0].content.parts[0].function_call

# Manual Pattern (When we reconstruct history)

if tool_call and tool_call.name == "get_current_temperature":
    result = get_current_temperature(**tool_call.args)
    print(f"Function execution result: {result}")

    # FIX: send function result back in same conversation
    followup_response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        types.Content(
            role="user",
            parts=[types.Part(text="What's the temperature in Bengaluru?")]
        ),
        types.Content(
            role="model",
            parts=[types.Part(function_call=tool_call)]
        ),
        types.Content(
            role="function",
            parts=[types.Part(
                function_response=types.FunctionResponse(
                    name=tool_call.name,
                    response=result
                )
            )]
        )
    ],
)
    print("Model’s natural language reply:")
    print(followup_response.candidates[0].content.parts[0].text)

else:
    print("No function call found in the response.")


Function execution result: {'location': 'Bengaluru', 'temperature': '21°C'}
Model’s natural language reply:
The current temperature in Bengaluru is 21°C.


In [2]:
# Function calling with thinking
from google import genai
from google.genai import types

client = genai.Client()

grounding_tool = types.Tool(
    google_search=types.GoogleSearch()
)

config = types.GenerateContentConfig(
    tools=[grounding_tool],
    thinking_config=types.ThinkingConfig(
        include_thoughts=True  # ask model to return reasoning so we know how the model thinks to pick the function/tools
    ),
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="List top 10 restaurants in bengaluru which is available in  today(10/05/2025) ",
    config=config,
    
)

print(response.text)


To provide the most accurate list of top 10 restaurants in Bengaluru open today, October 5, 2025, I need to check the current operating status of highly-rated establishments. Based on recent culinary reviews and awards for 2025, several restaurants are consistently mentioned for their exceptional dining experiences.

Here's a preliminary list of highly-regarded restaurants in Bengaluru from recent 2025 lists that I will now check for today's availability:

*   **NAVU**
*   **Farmlore**
*   **Copitas**
*   **Jamavar at The Leela Palace**
*   **Karavalli**
*   **ZLB23**
*   **Naru Noodle Bar**
*   **Bengaluru Oota Company**
*   **Comal**
*   **Marseli**
*   **Mirth**
*   **By The Blue**
*   **Vesparo X Pulse**
*   **Zarqash**
*   **The Fatty Bao**
*   **Black Pearl**
*   **Hi Seoul**
*   **Lupa**
*   **Vidyarthi Bhavan**
*   **Shiro**
*   **JW Kitchen**
*   **Kebabs and Kurries**
*   **Rim Naam**
*   **Kokoro**
*   **Hunaaan**
*   **Dali & Gala**

I will now proceed to verify the operati

In [3]:
#Inspecting Thought Signatures- mostly it will be displayed when we make function calling, to tell the model what operation performed in before request



import base64
# After receiving a response from a model with thinking enabled
# response = client.models.generate_content(...)

# The signature is attached to the response part containing the function call


for idx, part in enumerate(response.candidates[0].content.parts):
    if part.thought_signature:
        print(f"Found thought signature in part {idx}: {part.thought_signature}")

        print(base64.b64encode(part.thought_signature).decode("utf-8"))


In [29]:
# Function calling with thinking
from google import genai
from google.genai import types
from pydantic import BaseModel

class cost(BaseModel):
    dish_cost: int
client = genai.Client()


def bhai_biryani(dish:str):
    """ return the cost of the dish, where the dish and it's cost is stored as key value pair in menu dictionary"""
    menu = {'chicken fried rice':100, 'biryani':90, 'half kabab':50}
    return menu.get(dish,0)
def KFC(dish:str):
    """ return the dish cost(key value)"""
    menu={'biryani':200,'fried chicken':600}
    return menu.get(dish)

bhai_biryani_decl = types.FunctionDeclaration.from_callable(callable=bhai_biryani, client=client)
KFC_decl = types.FunctionDeclaration.from_callable(callable=KFC, client=client)

func_tools = types.Tool(function_declarations=[bhai_biryani_decl, KFC_decl])


config = types.GenerateContentConfig(
    tools=[func_tools],
    thinking_config=types.ThinkingConfig(
        include_thoughts=True  
    ),
    
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    #contents="what is the total cost if i order two biryani from bhai_biryani  ",
    contents="what is the cost of  biryani in bhai biryani  ",
    config=config,
    
)


for part in response.candidates[0].content.parts:
    if part.function_call:
        func_name = part.function_call.name
        func_args = part.function_call.args
        # Call the corresponding Python function manually
        if func_name == "bhai_biryani":
            result_value = bhai_biryani(**func_args)
        elif func_name == "KFC":
            result_value = KFC(**func_args)
        
        # Wrap in Pydantic model for structured access
        result = cost(dish_cost=result_value)
        print(result.dish_cost)  


90


In [16]:

# parallel function calling

from google import genai
from google.genai import types

def vegeterian(dish:str)->int:
    """ Get the rate of the vegeterian dish

    Args:
        dish : name of the vegeterian dish, e.g: sambar rice

    Returns:
        dictionary where the value is rate
    """
    menu ={'curd rice':50,'sambar rice':50, 'parotta':40}
    return {"rate ": menu.get(dish,0)}


def non_vegeterian(dish:str)->int:
    """ Get the rate of the non vegeterian dish

    Args:
        dish : name of the non vegeterian dish, e.g: chicken rice

    Returns:
        dictionary where the value is rate
    """
    menu ={'chicken rice':50,'mutton rice':50, 'beef rice':40}
    return {"rate ": menu.get(dish,0)}

client = genai.Client()

"""
# configure function calling mode, by default AUTO, ANY - prefer functions first
tool_config = types.ToolConfig(
    function_calling_config = types.FunctionCallingConfig(
        mode = 'ANY', allowed_function_names = ["vegeterian","non_vegeterian"]
    )
)
"""
config = types.GenerateContentConfig(

    #passing directly the function itself
    tools = [vegeterian,non_vegeterian],
    #tool_config = tool_config,
    
)


response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = "what is the rate of beef rice and curd rice",
    config = config,
)

print(response.text)

The rate of beef rice is 40 and curd rice is 50.


In [17]:
response

GenerateContentResponse(
  automatic_function_calling_history=[
    UserContent(
      parts=[
        Part(
          text='what is the rate of beef rice and curd rice'
        ),
      ],
      role='user'
    ),
    Content(
      parts=[
        Part(
          function_call=FunctionCall(
            args={
              <... Max depth ...>: <... Max depth ...>
            },
            name='non_vegeterian'
          ),
          thought_signature=b'\n\xf6\x03\x01\xd1\xed\x8ao\x10\xbb\x10W\xcf\xab\x18\t\xf3d\x93\x1b"%\x1d\xf0\x10[i\x0bz\x9c\x95\x1cJ\xa3\x81\x9a#\xc5\xc5\xe0\x1c\x0c+p\x06\xa8\x18\xe2\xa7Cw lh\t\xcc8\xf0\xbe\xd3\xf5/\xc4\x81\xf96\xe5\xa0qy\xfa\x80t@@\xc2\x9d\xcf\x0b_F\x94\xd0n\\Nw\xd8\xfd\xbeW\xb530\xf4\xd5\x04...'
        ),
        Part(
          function_call=FunctionCall(
            args={
              <... Max depth ...>: <... Max depth ...>
            },
            name='vegeterian'
          )
        ),
      ],
      role='model'
    ),
    Content(
 

In [31]:
# parallel function calling - how can i add structured output(pydantic) in this code:

from google import genai
from google.genai import types
from pydantic import BaseModel

class dishRate(BaseModel):
    dish:str
    rate:int

def vegeterian(dish:str)->int:
    """ Get the rate of the vegeterian dish

    Args:
        dish : name of the vegeterian dish, e.g: sambar rice

    Returns:
        dishRate object which contains dish name and dish rate
    """
    menu ={'curd rice':50,'sambar rice':50, 'parotta':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))


def non_vegeterian(dish:str)->int:
    """ Get the rate of the non vegeterian dish

    Args:
        dish : name of the non vegeterian dish, e.g: chicken rice

    Returns:
        dishRate object which contains dish name and dish rate
    """
    menu ={'chicken rice':50,'mutton rice':50, 'beef rice':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))

client = genai.Client()

"""
# configure function calling mode, by default AUTO, ANY - prefer functions first
tool_config = types.ToolConfig(
    function_calling_config = types.FunctionCallingConfig(
        mode = 'ANY', allowed_function_names = ["vegeterian","non_vegeterian"]
    )
)
"""
# for strutured output the function declaration is optimal(if i provide here, automatic function calling is not take place)
veg_decl = types.FunctionDeclaration.from_callable(callable = vegeterian, client=client)
nonveg_decl = types.FunctionDeclaration.from_callable(callable = non_vegeterian,client=client)
tools = types.Tool(function_declarations=[veg_decl,nonveg_decl])

config = types.GenerateContentConfig(
    
    #tools = [tools],
    tools = [vegeterian,non_vegeterian],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(),
    thinking_config=types.ThinkingConfig(include_thoughts=True),
    response_schema = list[dishRate],
    #response_mime_type = "application/json",
    
    #tool_config = tool_config,
    
)


response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = "what is the rate of beef rice and curd rice",
    config = config,
)

"""
print(response)
print(response.text)
structured_result: list[dishRate] = response.parsed
for dish in structured_result:
    print(f"{dish.dish}: {dish.rate}")
"""
print(response.text)
results = []

# iterate over all automatic_function_calling_history items
for history_item in response.automatic_function_calling_history:
    if hasattr(history_item, "parts"):
        for part in history_item.parts:
            if part.function_response:
                func_result = part.function_response.response.get("result")
                if func_result:
                    results.append(func_result)

# results now contains DishRate objects
for dish in results:
    print(f"{dish.dish}: {dish.rate}")



The rate of beef rice is 40 and curd rice is 50.
beef rice: 40
curd rice: 50


In [35]:
# parallel function calling 

from google import genai
from google.genai import types
from pydantic import BaseModel
import asyncio
import nest_asyncio
nest_asyncio.apply()

class dishRate(BaseModel):
    dish:str
    rate:int
class allRates(BaseModel):
    dishes:list[dishRate]

def vegeterian(dish:str)->dishRate:
    """ Get the rate of the vegeterian dish
    """
    menu ={'curd rice':50,'sambar rice':50, 'parotta':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))


def non_vegeterian(dish:str)->dishRate:
    """ Get the rate of the non vegeterian dish
    """
    menu ={'chicken rice':50,'mutton rice':50, 'beef rice':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))

client = genai.Client()




# configure function calling mode, by default AUTO, ANY - prefer functions first
tool_config = types.ToolConfig(
    function_calling_config = types.FunctionCallingConfig(mode = 'ANY')
)

# for strutured output the function declaration is optimal(if i provide here, automatic function calling is not take place)
#veg_decl = types.FunctionDeclaration.from_callable(callable = vegeterian, client=client)
#nonveg_decl = types.FunctionDeclaration.from_callable(callable = non_vegeterian,client=client)
#tool = types.Tool(function_declarations=[veg_decl,nonveg_decl])
async def main():
    config = types.GenerateContentConfig(
        
        #tools = [tool],
        tools = [vegeterian,non_vegeterian],
        automatic_function_calling=types.AutomaticFunctionCallingConfig( ),
        thinking_config=types.ThinkingConfig(include_thoughts=True),
        #response_schema = allRates,
        #response_mime_type = "application/json",
        response_schema=allRates,
        tool_config = types.ToolConfig(
            function_calling_config=types.FunctionCallingConfig(mode="AUTO")
        ),
        
    )

    response = await client.aio.models.generate_content(
        model="gemini-2.5-flash",
        #contents="what is the rate of beef rice(non veg) and curd rice(veg)",
        contents ="what is the rate of beef rice(non veg) and curd rice(veg)" ,

        config=config,
    )
    structured_result: allRates = response.parsed
    if response.parsed:
        for dish in response.parsed.dishes:
            print(f"{dish.dish}: {dish.rate}")
    else:
        print("Model did not return a structured response.")
        print(response.text)
    


await main()



Model did not return a structured response.
The rate of beef rice is 40 and curd rice is 50.


In [53]:
# compositional function calling

from google import genai
from google.genai import types
import json
def pizza_bakery_mani(flavour:str)->dict:
    """ Get the price for a given given flavour"""
    menu = {'chicken':100,'tomato':70,'corn':50,'beef':80}
    return {'flavour':flavour,'price':menu.get(flavour)}

def pizza_bakery_mani_gst(price:int)->dict:
    """ return the gst for this price"""
    return {"gst_price":price*0.10}

client = genai.Client()
config = types.GenerateContentConfig(
    tools=[pizza_bakery_mani, pizza_bakery_mani_gst ]
)


response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Get me the price and gst for chicken(flavour) pizza from pizza bakery mani",
    config=config,
)

print(response.text)


The price for chicken pizza is 100 and the GST for it is 10.


In [4]:
# parallel function calling 

from google import genai
from google.genai import types
from pydantic import BaseModel
import asyncio


class dishRate(BaseModel):
    dish:str
    rate:int
class allRates(BaseModel):
    dishes:list[dishRate]

def vegeterian(dish:str)->int:
    """ Get the rate of the vegeterian dish
    """
    menu ={'curd rice':50,'sambar rice':50, 'parotta':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))


def non_vegeterian(dish:str)->int:
    """ Get the rate of the non vegeterian dish
    """
    menu ={'chicken rice':50,'mutton rice':50, 'beef rice':40}
    return dishRate(dish=dish, rate = menu.get(dish,0))

client = genai.Client()




# configure function calling mode, by default AUTO, ANY - prefer functions first
tool_config = types.ToolConfig(
    function_calling_config = types.FunctionCallingConfig(mode = 'ANY')
)

# for strutured output the function declaration is optimal(if i provide here, automatic function calling is not take place)
#veg_decl = types.FunctionDeclaration.from_callable(callable = vegeterian, client=client)
#nonveg_decl = types.FunctionDeclaration.from_callable(callable = non_vegeterian,client=client)
#tool = types.Tool(function_declarations=[veg_decl,nonveg_decl])

config = types.GenerateContentConfig(
    
    #tools = [tool],
    tools = [vegeterian,non_vegeterian],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(),
    thinking_config=types.ThinkingConfig(include_thoughts=True),
    response_schema = allRates,
    #response_mime_type = "application/json",
    
    tool_config = types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="ANY")
    ),
    
)

"""
response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = "what is the rate of beef rice(non veg) and curd rice(veg)",
    config = config,
)
"""
async def main():
    response = await client.aio.models.generate_content(
        model="gemini-2.5-flash",
        contents="what is the rate of beef rice(non veg) and curd rice(veg)",
        config=config,
    )

    print("Automatic Function Calls:\n", response.automatic_function_calling_history)
    
    print(response)
    #print(response.text)
    structured_result: allRates = response.parsed
    for dish in structured_result.dishes:
        print(f"{dish.dish}: {dish.rate}")



asyncio.run(main())
"""
print(response.text)
results = []

# iterate over all automatic_function_calling_history items
for history_item in response.automatic_function_calling_history:
    if hasattr(history_item, "parts"):
        for part in history_item.parts:
            if part.function_response:
                func_result = part.function_response.response.get("result")
                if func_result:
                    results.append(func_result)

# results now contains DishRate objects
for dish in results:
    print(f"{dish.dish}: {dish.rate}")

"""

RuntimeError: asyncio.run() cannot be called from a running event loop

In [37]:
import os
from google import genai
from google.genai import types
from pydantic import BaseModel

# -----------------------------
# 1️⃣ Define Pydantic schema
# -----------------------------
class ThermostatResult(BaseModel):
    location: str
    current_temperature: int
    thermostat_setting: int
    status: str

# -----------------------------
# 2️⃣ Define functions
# -----------------------------
def get_weather_forecast(location: str) -> dict:
    """Gets the current weather temperature for a given location."""
    print(f"Tool Call: get_weather_forecast(location={location})")
    print("Tool Response: {'temperature': 25, 'unit': 'celsius'}")
    return {"temperature": 25, "unit": "celsius"}  # Dummy response

def set_thermostat_temperature(temperature: int) -> dict:
    """Sets the thermostat to a desired temperature."""
    print(f"Tool Call: set_thermostat_temperature(temperature={temperature})")
    print("Tool Response: {'status': 'success'}")
    return {"status": "success"}

# -----------------------------
# 3️⃣ Configure client + schema
# -----------------------------
client = genai.Client()

config = types.GenerateContentConfig(
    tools=[get_weather_forecast, set_thermostat_temperature],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(),
    response_schema=ThermostatResult,   # 👈 NEW: structured output type
)

# -----------------------------
# 4️⃣ Run model
# -----------------------------
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="If it's warmer than 20°C in London, set the thermostat to 20°C, otherwise set it to 18°C.",
    config=config,
)

# -----------------------------
# 5️⃣ Access structured output
# -----------------------------
if response.parsed:
    result: ThermostatResult = response.parsed
    print("✅ Structured Result:")
    print(result.json(indent=2))

    # You can reuse later like:
    print("\nLater use example:")
    print(f"Thermostat was set to {result.thermostat_setting}°C in {result.location}")
else:
    print("Model did not return structured output.")
    print(response.text)


Tool Call: get_weather_forecast(location=London)
Tool Response: {'temperature': 25, 'unit': 'celsius'}


Tool Call: set_thermostat_temperature(temperature=20)
Tool Response: {'status': 'success'}
Model did not return structured output.
I have set the thermostat to 20°C.


In [22]:
from google import genai
from google.genai import types

client = genai.Client()

# -------------------------------
# Step 1: Define the function
# -------------------------------
restaurant_rating_function = {
    "name": "get_restaurant_rating",
    "description": "Fetches the average rating for a given restaurant in Bengaluru.",
    "parameters": {
        "type": "object",
        "properties": {
            "restaurant_name": {
                "type": "string",
                "description": "The name of the restaurant.",
            },
        },
        "required": ["restaurant_name"],
    },
}

def get_restaurant_rating(restaurant_name: str):
    # Mock data – in real case, this could query Zomato/Google Maps API
    mock_ratings = {
        "Farmlore": 4.8,
        "Jamavar": 4.6,
        "Karavalli": 4.7,
        "ZLB23": 4.5,
        "Copitas": 4.6,
        "The Fatty Bao": 4.4,
        "Rim Naam": 4.7,
        "Mavalli Tiffin Room": 4.3,
        "Yuki Cocktail Bar & Kitchen": 4.5,
        "Lotus Pavilion": 4.4,
    }
    return {"restaurant": restaurant_name, "rating": mock_ratings.get(restaurant_name, "N/A")}

# -------------------------------
# Step 2: Configure with thinking
# -------------------------------
tools = types.Tool(function_declarations=[restaurant_rating_function])

config = types.GenerateContentConfig(
    tools=[tools],
    thinking_config=types.ThinkingConfig(
        include_thoughts=True  # Ask the model to return its reasoning
    ),
)

# -------------------------------
# Step 3: Ask the model
# -------------------------------
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is the rating of Farmlore restaurant in Bengaluru?",
    config=config,
)

# -------------------------------
# Step 4: Inspect response
# -------------------------------
for idx, part in enumerate(response.candidates[0].content.parts):
    if part.thought_signature:
        print(f"Found thought signature in part {idx}: {part.thought_signature}")

    if part.function_call:
        print(f"\nModel requested function: {part.function_call.name}")
        print(f"Arguments: {part.function_call.args}")

        # Execute the function
        result = get_restaurant_rating(**part.function_call.args)
        print(f"Function execution result: {result}")

        # -------------------------------
        # Step 5: Send function result back
        # -------------------------------
        followup_response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                response.candidates[0].content,  # include model’s function call turn
                types.Content(
                    role="function",
                    parts=[types.Part(
                        function_response=types.FunctionResponse(
                            name=part.function_call.name,
                            response=result
                        )
                    )]
                )
            ],
        )

        print("\nFinal model reply:")
        print(followup_response.text)


Found thought signature in part 1: b'\n\xa7\x02\x01\xd1\xed\x8ao\x83\x12\x97\xea+\xfc5,\xe1:\xbd\xd2T-\x04Ux\'\x8cLbs\x96\xff\xcaT\x03\\\x9f\xd7\x93=\nTM\x93\x01OgNS\\\x0e\xa7\xd1\x02c\xea\x1e\xd9\x06\xac\xb0~\x93\xae\xe2^\x85L=\xb0\xcd\x9a\x08x\x91\xd7\xa4F\xa6e\xf2\xe2\xc4\x02\xa4\xf4\xf0\x13P\x8e\xbd\x88TL\xdef\xa9\x8aW&\xbb\x15\xad\x87g\xb1\x10\x87\xacM\xe1\xaed2T\x84\x98.\x16\xa5\xc8\xd5\x0bQ\x91\xa0;\xb3\xa0\x06\xdc\x08\xec\xb7\x98\xde\xf9\xd2\x00#\xb7\xe6\x82\xfb"\xbej\x9dyFY\x97\x94\xddg_\xa6V.f\xf4\xf0p\xce\x8dt\x92!\xb159D\x16\xfb\xac\xa8\xc9\x05\xf8\xb3\xf3~\x11!D-\xad\x04\xf1rY\xa4H\xe2\x804\x1e\x01\xba:RU\xe4\xe7\x9d\xc7G\xd8-(\x0e`\xb6\x9fM\x18\xbd\x86\xe1py.-\xe5\xbc\x8a\xcd;\xdc7\xeb^\r\xbf\x99\x05K\x8b]\x8b\xbb4\xe9\xa8\x98<*8l\x9e\xbfd{\xead\x14\x08\xd3\xa3\x1b\x83v\x92H\xaa\xc9Ox\x1a\xb2_\xd6.\x03\x89\xd0>\t\x08E\xd2C\xa4\xaa\xfeQ\x97\xff\x1e\x8c\x9a\xf5\x97p\xfa,L3'

Model requested function: get_restaurant_rating
Arguments: {'restaurant_name': 'Farmlore'}
Function 

In [6]:
# Automatic function schema declaration

from google import genai
from google.genai import types

def multiply(a: float, b: float):
    """Returns a * b."""
    return a * b

client = genai.Client()

#  FunctionDeclarations from Python functions directly using types.FunctionDeclaration.from_callable(client=client, callable=your_function).
fn_decl = types.FunctionDeclaration.from_callable(callable=multiply, client=client)

print(fn_decl)
# to_json_dict() provides a clean JSON representation.
print(fn_decl.to_json_dict())

behavior=None description='Returns a * b.' name='multiply' parameters=Schema(
  properties={
    'a': Schema(
      type=<Type.NUMBER: 'NUMBER'>
    ),
    'b': Schema(
      type=<Type.NUMBER: 'NUMBER'>
    )
  },
  required=[
    'a',
    'b',
  ],
  type=<Type.OBJECT: 'OBJECT'>
) parameters_json_schema=None response=None response_json_schema=None
{'description': 'Returns a * b.', 'name': 'multiply', 'parameters': {'properties': {'a': {'type': 'NUMBER'}, 'b': {'type': 'NUMBER'}}, 'required': ['a', 'b'], 'type': 'OBJECT'}}


In [ ]:
# Multi-tool use: Combine native tools with function calling

